In [32]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [44]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'd-vp-model'
id_name = "Id"
target = 'WnvPresent'

file_name = "./d-vp/test_West-Nile Virus Prediction.csv"
test_file_name = "./d-vp/test_West-Nile-Virus-Prediction-no-label.csv"

kaggle_file_name = "./d-vp/kaggle-test-West Nile Virus Prediction.csv"
kaggle_test_file_name = "./d-vp/kaggle-test-West Nile Virus Prediction-no-label.csv"
kaggle_pred_file_name = "./d-vp/kaggle-test-West Nile Virus Prediction-predictions.csv"

         Date                                            Address  \
0  2007-05-29   1100 South Peoria Street, Chicago, IL 60608, USA   
1  2013-09-12  1300 North Laramie Avenue, Chicago, IL 60651, USA   
2  2011-09-23      3500 West 51st Street, Chicago, IL 60632, USA   
3  2013-08-22                                   Chicago, IL, USA   
4  2007-08-15                  West 37th Place, Chicago, IL, USA   

                  Species  Block                   Street  Trap  \
0          CULEX RESTUANS     11              S PEORIA ST  T091   
1           CULEX PIPIENS     13            N LARAMIE AVE  T230   
2  CULEX PIPIENS/RESTUANS     35                W 51ST ST  T062   
3  CULEX PIPIENS/RESTUANS     43   N ASHLAND AVE OVERPASS  T145   
4           CULEX PIPIENS     81               W 37TH PL.  T157   

                      AddressNumberAndStreet   Latitude  Longitude  \
0             1100  S PEORIA ST, Chicago, IL  41.862292 -87.648860   
1           1300  N LARAMIE AVE, Chicago, IL  41

In [34]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)

df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

         Date                                            Address  \
0  2009-09-17       South Brandon Avenue, Chicago, IL 60617, USA   
1  2011-08-19  2800 North Francisco Avenue, Chicago, IL 60618...   
2  2007-08-24   8000 South Kedzie Avenue, Chicago, IL 60652, USA   
3  2007-06-29       4200 West 127th Street, Alsip, IL 60803, USA   
4  2007-07-25        South Stony Island Avenue, Chicago, IL, USA   

                  Species  Block               Street  Trap  \
0  CULEX PIPIENS/RESTUANS     13            S BRANDON  T209   
1           CULEX PIPIENS     28      N FRANCISCO AVE  T147   
2  CULEX PIPIENS/RESTUANS     80         S KEDZIE AVE  T072   
3          CULEX RESTUANS     42           W 127TH PL  T135   
4           CULEX PIPIENS     10   S STONY ISLAND AVE  T138   

                  AddressNumberAndStreet   Latitude  Longitude  \
0           1300  S BRANDON, Chicago, IL  41.740641 -87.546587   
1     2800  N FRANCISCO AVE, Chicago, IL  41.932094 -87.700117   
2        8000 

In [35]:
f = open(test_file_name)
count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    #print(response)
    pred_label.append(int(response[0]))
    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    #print(neg_prob)
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    #print(pos_prob)
    pred_pos_prpb.append(pos_prob)    


In [41]:
print(pred_label[:10])
print(pred_neg_prpb[:10])
print(pred_pos_prpb[:10])

import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print(np.mean(y_true==y_pred))

from sklearn import metrics
#fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_pos_prpb, pos_label=1)
#metrics.auc(fpr, tpr)
metrics.roc_auc_score(y_true, pred_pos_prpb)

[0, 0, 0, 0, 0, 1, 0, 0, 1, 1]
[0.9934498285874724, 0.9639562778174877, 0.9821963366121054, 0.9998925271211192, 0.9919961951673031, 0.3323650360107422, 0.6213184595108032, 0.9995773318223655, 0.48769092559814453, 0.4396812915802002]
[0.006550171412527561, 0.03604372218251228, 0.01780366338789463, 0.00010747287888079882, 0.008003804832696915, 0.6676349639892578, 0.3786815404891968, 0.0004226681776344776, 0.5123090744018555, 0.5603187084197998]
0.0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


0.9273290154385107

In [42]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)
kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name], axis=1) 
df['NumMosquitos'] = 0
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

         Date                                            Address  \
0  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
1  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
2  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
3  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
4  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   

                  Species  Block           Street  Trap  \
0  CULEX PIPIENS/RESTUANS     41   N OAK PARK AVE  T002   
1          CULEX RESTUANS     41   N OAK PARK AVE  T002   
2           CULEX PIPIENS     41   N OAK PARK AVE  T002   
3        CULEX SALINARIUS     41   N OAK PARK AVE  T002   
4         CULEX TERRITANS     41   N OAK PARK AVE  T002   

              AddressNumberAndStreet  Latitude  Longitude  AddressAccuracy  \
0  4100  N OAK PARK AVE, Chicago, IL  41.95469 -87.800991                9   
1  4100  N OAK PARK AVE, Chicago, IL  41.95469 -87.800991                9   
2 

In [43]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '0c045d29-f55a-4de9-802c-ed1ded3d7b40',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0c045d29-f55a-4de9-802c-ed1ded3d7b40',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 23 Mar 2021 16:56:51 GMT'},
  'RetryAttempts': 0}}